# User interface Node #

In [1]:
#! /usr/bin/env python3

import rospy
import actionlib
import actionlib.msg
from assignment_2_2022.srv import Ngoal
import assignment_2_2022.msg
import time
import sys
import select
import os   
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import logging
from IPython.display import display

act_clnt = None
goal = assignment_2_2022.msg.PlanningGoal() 
goal.target_pose.pose.position.z = 0.0
info = Ngoal()
srv_info = None
gf = 0 # goal flag to increase the number of goal only the first time the state pass to succeeded
n_goal = 0 # number of goal reached
n_deleted = 0 # number of goal deleted

In [2]:
def D_cb(b):
    print("2")
    if ((act_clnt.get_state()) == 1):
        act_clnt.cancel_goal() # delete the goal
        n_deleted = rospy.get_param('n_deleted') # get the actual number of goal deleted
        n_deleted = n_deleted + 1 # increase it
        rospy.set_param('n_deleted', n_deleted) # load it in the paramserver

In [3]:
def N_cb(b):
    print("N")
    if ((Gx.disabled == True) and (Gy.disabled == True)):
        Gx.disabled = False
        Gy.disabled = False     
        N.disabled = True

In [4]:
def NGx_cb(value):   
    print("Gx")
    Gx.disabled = True
    goal.target_pose.pose.position.x = float(Gx.value)
    if (Gy.disabled):
        N.disabled = False
        act_clnt.send_goal(goal)
    

In [5]:
def NGy_cb(value):    
    Gy.disabled = True
    goal.target_pose.pose.position.y = float(Gy.value)
    if (Gx.disabled):
        N.disabled = False  
        act_clnt.send_goal(goal)

In [6]:
def I_cb(b):
    info = srv_info()
    Gr.value = info.n_goal
    Gd.value = info.n_deleted

In [7]:
def UI(): 
    global D, N, I, Gx, Gy, Gr, Gd
    print("ui")
    
    # different layout:
    L1 = Layout(width = 'auto', height = '10%', min_width = '80px', min_height = '54px', max_width = '160px')
    L2 = Layout(width = 'auto', height = '5%', min_width = '40px', min_height = '20px', max_width = '80px')
    
    # delete goal button:
    D = widgets.Button(value = False, description = 'Delete Goal', disabled = False, button_style = 'danger', layout = L1, continuous_update = False)
    
    # new goal button:
    N = widgets.Button(value = False, description = 'New Goal', disabled = False, button_style = 'success', layout = L1, continuous_update = False)
    
    # information button:
    I = widgets.Button(value = False, description = 'Information', disabled = False, button_style = 'info', layout = L1, continuous_update = False)
    
    # info box for the goal coordinates:
    # name:
    Gx_n = widgets.Text(value = 'x_goal', disabled = True, layout = L2)
    Gy_n = widgets.Text(value = 'y_goal', disabled = True, layout = L2)
    # data:
    Gx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gx_c = VBox([Gx_n, Gx])
    Gy_c = VBox([Gy_n, Gy])
    Gxy = HBox([Gx_c, Gy_c])
    
    
    # info box for the goal reached and deleted: FloatText
    # name:
    Gr_n = widgets.Text(value = 'goal_r', disabled = True, layout = L2)
    Gd_n = widgets.Text(value = 'goal_d', disabled = True, layout = L2)
    # data:
    Gr = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gd = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gr_c = VBox([Gr_n, Gr])
    Gd_c = VBox([Gd_n, Gd])
    Grd = HBox([Gr_c, Gd_c])
    
    # combine everything
    UI = VBox([D, HBox([N, Gxy]), HBox([I, Grd])])
    
    # constraints on the dimensions: 0.00
    
    # button callbacks:
    D.on_click(D_cb)
    N.on_click(N_cb)
    # when i've typed what needed in the text boxes call the cb NG_cb()
    Gx.observe(NGx_cb, names = 'value')
    Gy.observe(NGy_cb, names = 'value')
    I.on_click(I_cb)
    
    print("UI")
    display(UI)


In [8]:
def init():
    print("init")
    global goal, act_clnt, info, srv_info
        
    rospy.init_node('user_interface', anonymous=True) # init the node
    act_clnt = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
    act_clnt.wait_for_server()
    srv_info = rospy.ServiceProxy("/info", Ngoal)
    rospy.wait_for_service("/info")
    
    rospy.set_param('n_goal', n_goal) # setting the parameters in the paramserver
    rospy.set_param('n_deleted', n_deleted) # setting the parameters in the paramserver

In [9]:
def check_goal():
    print("cg")

    
    if (act_clnt.get_state() == 3 and gf == 0): # if i've reached the goal
        n_goal = rospy.get_param('n_goal') # get the actual number of goal
        n_goal = n_goal + 1 # increase it
        rospy.set_param('n_goal', n_goal) # load it in the paramserver
        gf = 1
    elif (act_clnt.get_state() != 3):
        gf = 0
    else:
        return
    

In [10]:
def main():
    print("a")
    init()
    print("b")
    check_goal()
    print("c")
    UI()
    print("ad")
    rate = rospy.Rate(20)
    print("rr")
main()

a
init
b
cg
c
ui
UI


ad
rr
N
Gx
2
